<a href="https://colab.research.google.com/github/Makoto-Noda/RSNA2023/blob/main/RSNA2023_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import pydicom
from pydicom.data import get_testdata_files
import plotly.express as px
from pprint import pprint
import nibabel as nib
from ipywidgets import interact

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import json
f = open("/content/drive/MyDrive/kaggle.json", 'r')
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

In [10]:
!kaggle competitions download -c rsna-2023-abdominal-trauma-detection -f train.csv

100% 104k/104k [00:00<00:00, 1.04MB/s]
100% 104k/104k [00:00<00:00, 1.04MB/s]


In [11]:
train = pd.read_csv("train.csv")
train.head()

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1


In [13]:
print("Number of rows and columns:", train.shape)
print()
print('*************************************')
print()
print(train.info())
print()
print('*************************************')
print()
print("Number of Unique Values:\n",train.nunique())
print()
print('*************************************')
print()
print("Duplicate Values:\n",train.duplicated().sum())
print()
print('*************************************')
print()
print("Null Values:\n",train.isnull().sum())

Number of rows and columns: (3147, 15)

*************************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3147 entries, 0 to 3146
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   patient_id             3147 non-null   int64
 1   bowel_healthy          3147 non-null   int64
 2   bowel_injury           3147 non-null   int64
 3   extravasation_healthy  3147 non-null   int64
 4   extravasation_injury   3147 non-null   int64
 5   kidney_healthy         3147 non-null   int64
 6   kidney_low             3147 non-null   int64
 7   kidney_high            3147 non-null   int64
 8   liver_healthy          3147 non-null   int64
 9   liver_low              3147 non-null   int64
 10  liver_high             3147 non-null   int64
 11  spleen_healthy         3147 non-null   int64
 12  spleen_low             3147 non-null   int64
 13  spleen_high            3147 non-null   int64
 14  any_injur

In [14]:
!kaggle competitions download -c rsna-2023-abdominal-trauma-detection -f train_series_meta.csv

  0% 0.00/90.7k [00:00<?, ?B/s]
100% 90.7k/90.7k [00:00<00:00, 1.67MB/s]


In [16]:
train_series_meta = pd.read_csv("train_series_meta.csv")
train_series_meta.head()

,patient_id,series_id,aortic_hu,incomplete_organ
0,10004,21057,146.00,0
1,10004,51033,454.75,0
2,10005,18667,187.00,0
3,10007,47578,329.00,0
4,10026,29700,327.00,0


In [17]:
print("Number of rows and columns:", train_series_meta.shape)
print()
print('*************************************')
print()
print(train_series_meta.info())
print()
print('*************************************')
print()
print("Number of Unique Values:\n",train_series_meta.nunique())
print()
print('*************************************')
print()
print("Duplicate Values:\n",train_series_meta.duplicated().sum())
print()
print('*************************************')
print()
print("Null Values:\n",train_series_meta.isnull().sum())

Number of rows and columns: (4711, 4)

*************************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4711 entries, 0 to 4710
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   patient_id        4711 non-null   int64  
 1   series_id         4711 non-null   int64  
 2   aortic_hu         4711 non-null   float64
 3   incomplete_organ  4711 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 147.3 KB
None

*************************************

Number of Unique Values:
 patient_id          3147
series_id           4711
aortic_hu            717
incomplete_organ       2
dtype: int64

*************************************

Duplicate Values:
 0

*************************************

Null Values:
 patient_id          0
series_id           0
aortic_hu           0
incomplete_organ    0
dtype: int64
